In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/Data.xlsx')

df.drop(df.index[(df["Sınıf"] == "Safra kesesi taşı")],axis=0,inplace=True)
'''
df.drop(df.index[(df["Sınıf"] == "Kolon")],axis=0,inplace=True)  
df.drop(df.index[(df["Sınıf"] == "Pankreas")],axis=0,inplace=True)
df.drop(df.index[(df["Sınıf"] == "Böbrek-Mesane")],axis=0,inplace=True)
df.drop(df.index[(df["Sınıf"] == "Abdominal Aorta")],axis=0,inplace=True)
df.drop(df.index[(df["Sınıf"] == "Safra Kesesi")],axis=0,inplace=True)
df.drop(df.index[(df["Sınıf"] == "Apandiks")],axis=0,inplace=True)
'''

df = df.reset_index(drop=True)
df.info

a = df.groupby('Sınıf').size()
print(a)
sizes = [1200,1200,1200,1200,1200,1200,1200]

!pip install pydicom


df['Sınıf'] = df['Sınıf'].map({'Akut apandisit ile uyumlu':1, 'Apendikolit':1, 
                               'Akut kolesistit ile uyumlu':2, 
                               'Akut pankreatit ile uyumlu':3,
                               'Böbrek taşı':4, 'Üreter taşı':4, 
                               'Akut divertikülit ile uyumlu':5, 'Kalsifiye divertikül':5, 
                               'Abdominal aort diseksiyon':6, 'Abdominal aort anevrizma':6,
                               'Kolon':0, 'Pankreas':0, 'Böbrek-Mesane':0, 'Abdominal Aorta':0,'Safra Kesesi':0, 'Apandiks':0 })

#Parsing Coordinates
def parse_coordinates(coordinate_string):
  if pd.isnull(coordinate_string):
    left_x,left_y,right_x,right_y = 0,0,0,0  
    
  else:
    left,right = coordinate_string.split('-')
    left_x,left_y = left.split(',')
    right_x,right_y = right.split(',')
    
  return np.array([int(left_x),int(left_y),int(right_x),int(right_y)],dtype=np.float16)


#Reading dicom images, corresponding labels and coordinates to that images
import zipfile 
import pydicom
import math
from collections.abc import Iterable
import pandas
x =[]
y =[]
coordinates = []
def read_dicom(img_path, colormap = None, extra_brightness=0):
  ds = pydicom.dcmread(img_path)
  shape = ds.pixel_array.shape
  target = 255

  # Convert to float to avoid overflow or underflow losses.
  image_2d = ds.pixel_array.astype(np.float16)
  img_data = image_2d
  multival = isinstance(ds.WindowCenter, Iterable)
  if multival:
      scale_center = -ds.WindowCenter[0]
  else:
      scale_center = -ds.WindowCenter
  intercept = scale_center+ds.RescaleIntercept+extra_brightness
  image_2d += intercept

  # Rescaling grey scale between 0-255
  image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
  print(f"after scaling to 0-255, min: {image_2d_scaled.min()}, max: {image_2d_scaled.max()}")

  
  # Convert to uint
  #image_2d_scaled = np.uint8(image_2d_scaled)
  return image_2d_scaled



training_zip = zipfile.ZipFile("/content/drive/MyDrive/Training.zip",mode='r')
print(zipfile)
with training_zip as zipfile:
  for index,row in df.iterrows():
    if sizes[row['Sınıf']] > 0:
      new_image_path = zipfile.extract('Training/'+str(row['Olgu Numarası'])+'/'+str(row['Kesit Numarası'])+'.dcm','/content/sample_data/images')
      print(new_image_path) 
      x.append(read_dicom(new_image_path)) 
      y.append(row['Sınıf'])
      coordinates.append(parse_coordinates(row['Koordinatlar']))
      sizes[row['Sınıf']] = sizes[row['Sınıf']] - 1  


print(str(len(x))+'----------------'+str(len(y))+'-----'+str(len(coordinates))) 
print(type(x))         
    

#Basically, you need to reshape your data to (n_images, x_shape, y_shape, channels).
import tensorflow as tf
import matplotlib.pyplot as plt

INPUT_SIZE = (512,512)

def preprocess_dataset(image,height,width,i):
  coordinates[i][0] = coordinates[i][0]*512/width
  coordinates[i][2] = coordinates[i][2]*512/width
  
  coordinates[i][1] = coordinates[i][1]*512/height
  coordinates[i][3] = coordinates[i][3]*512/height
    
  image = tf.image.resize(image, (INPUT_SIZE[0], INPUT_SIZE[1]))
  return (image)
for i in range(0,len(x)):
  x[i] =preprocess_dataset(x[i].reshape(x[i].shape[0],x[i].shape[1],1),x[i].shape[0],x[i].shape[1],i)


#scale the bounding box coordinates relative to the spatial
# dimensions of the input image

for i in range(0,len(coordinates)):
  coordinates[i] = coordinates[i]/512.0
print(coordinates[1])  

bboxes = np.array(coordinates)
print(bboxes.shape)

labels = np.array(y).reshape(8400,1)
print(labels.shape)




'''
import matplotlib.pyplot as plt
plt.Figure(figsize=(14,14))
plt.imshow(x[8399][:,:,-1])
'''

!pip install tensorflow
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

split = train_test_split(x, labels, bboxes,test_size=0.20, random_state=42)
# unpack the data split
(x_train, x_test) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]



# construct a dictionary for our target training outputs
trainTargets = {
	"class_label": np.array(trainLabels),
	"bounding_box": np.array(trainBBoxes)
}
# construct a second dictionary, this one for our target testing
# outputs
testTargets = {
	"class_label": np.array(testLabels),
	"bounding_box": np.array(testBBoxes)
}

import tensorflow as tf
from os import name
from keras.backend import shape
import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dropout,Dense,Flatten,MaxPooling2D,Conv2D,AveragePooling2D
#alexNetHEAD
alexNet = keras.Input(shape=(512,512,1))


alexNet1 = Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding="same")(alexNet)
alexNet2 = AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid')(alexNet1)

alexNet3 = Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid')(alexNet2)
alexNet4 = AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(alexNet3)

#alexNet6 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(alexNet5)

flattenLayer = (Flatten())(alexNet4)


# Compile the model
#model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='SGD', metrics=['accuracy'])
ardaNetModel = Model(alexNet,flattenLayer)

tf.keras.utils.plot_model(ardaNetModel)

from tensorflow.keras.layers import Dropout,Dense,Flatten
from tensorflow.keras import Model

'''bboxHeaddense1 = Dense(96, activation='relu')(flattenLayer)
bboxHeaddense2 = Dense(96, activation='relu')(bboxHeaddense1)
bboxHeaddense3 = Dense(96, activation='relu')(bboxHeaddense2)
bboxHeaddense4 = Dense(96, activation='relu')(bboxHeaddense3)
bboxHeaddense5 = Dense(96, activation='relu')(bboxHeaddense4)
bboxHeaddense6 = Dense(96, activation='relu')(bboxHeaddense5)
bboxHeaddense7 = Dense(96, activation='relu')(bboxHeaddense6)
bboxHeaddense8 = Dense(96, activation='relu')(bboxHeaddense7)
bboxHeaddense9 = Dense(4, activation="sigmoid",
	name="bounding_box")(bboxHeaddense8)'''
# construct a second fully-connected layer head, this one to predict
# the class label
softmaxdense1 = Dense(84, activation='tanh')(flattenLayer)
softmaxHead = Dense(7, activation="softmax",name="class_label")(softmaxdense1)
# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model = Model(
	inputs=alexNet,
	outputs=(bboxHeaddense9, softmaxHead))

losses = {
	"class_label": "SparseCategoricalCrossentropy",
	"bounding_box": "mean_squared_error",
}
model.compile(loss=losses, optimizer='adam', metrics=["accuracy"])
tf.keras.utils.plot_model(model)


'''from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
strtfdKfold = StratifiedKFold(n_splits = 10)
kfold = strtfdKfold.split()'''

history = model.fit(np.array(x_train), trainTargets,batch_size=64,validation_split=0.1,epochs=10,verbose=1)

#model.fit(np.array(x_train),np.array(y_train),batch_size = 64 ,epochs=20, validation_split = 0.1) 



#model.save('/content/drive/MyDrive/DeepLearningModel')

model.evaluate(np.array(x_test),testTargets,batch_size=64)

predictions = model.predict(np.array(x_test))

class_predictions = np.array(predictions[1])
bbox_predictions = np.array(predictions[0])

print(len(predictions[1][1]))

model = tf.keras.models.load_model('/content/drive/MyDrive/DeepLearningModel')

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 2, random_state = 42,max_features=4)

rf.fit(x_test, testBBoxes)

bbox_predictions = rf.predict(bbox_predictions)

class_predictions = np.argmax(class_predictions, axis=1)
print(class_predictions)


#Determining f1-scores
from sklearn.metrics import f1_score
f1_score(testLabels,class_predictions,average='weighted')

#Determining iou (intersection over union)
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

average = 0
for i in range(0,len(testBBoxes)):
  if bbox_predictions[i][0] == 0.0  or testBBoxes[i][0] == 0.0:
    continue
  iou = bb_intersection_over_union(testBBoxes[i]*512.0,bbox_predictions[i]*512.0)
  print('actual values :'+str(testBBoxes[i]*512)+'-----------Predicted Values:'+str(bbox_predictions[i]*512))
  average = average + iou
  print(iou)
print('average iou:')
print(average/len(testBBoxes))  

%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt 

def plot_confusion_matrix(cm,classes,normalize=False,title='Confusion Matrix',cmap=plt.cm.Blues):
  plt.imshow(cm,interpolation='nearest',cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks,classes,rotation=45)
  plt.yticks(tick_marks,classes)

  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
    print('Normalized Confusion Matrix')
  else:
    print('Confusion matrix without normalization')
  print(cm)

  thresh = cm.max()/2

  for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(j,i,cm[i,j],horizontalalignment='center',color='white' if cm[i,j]>thresh else 'black')
  plt.tight_layout()
  plt.ylabel("True Label")
  plt.xlabel("Predicted Label")


classes = ['Kolon,Pankreas,Böbrek-Mesane,Abdominal Aorta,Safra Kesesi,Apandiks',
          'Akut apandisit ile uyumlu,Apendikolit', 
          'Akut kolesistit ile uyumlu', 
          'Akut pankreatit ile uyumlu',
          'Böbrek taşı,Üreter taşı', 
          'Akut divertikülit ile uyumlu,Kalsifiye divertikül', 
          'Abdominal aort diseksiyon,Abdominal aort anevrizma']
cm = confusion_matrix(testTargets['class_label'],class_predictions)
plot_confusion_matrix(cm,classes,title='Confusion Matrix')           

def get_confusion_matrix_values(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    return(cm[0][0], cm[0][1], cm[1][0], cm[1][1])

TP, FP, FN, TN = get_confusion_matrix_values(x_test, x_pred)

acc = (TP+TN)/(TP+FP+FN+TN)

from sklearn.metrics import plot_confusion_matrix
fig, ax = plt.subplots(figsize=(15, 15))
plot_confusion_matrix(slf_4, X_test, y_test, normalize='true', cmap=plt.cm.Blues, ax=ax)
plt.show()